In [ ]:
# import '../config.py

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

#below needed to display decision tree
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
# from sklearn.metrics import mean_squared_error

import json
import requests
from bs4 import BeautifulSoup

#import my function files
import webscrape_wiki_table
import config
import query_helper
import time

#utilities file for timer
from utilities import *

/Users/gabrielseemann/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:

Timer.start()
Timer.end()

0.0 seconds elapsed


In [3]:
query_helper.create_db('mushroom')

In [4]:
mushroom_table = """
CREATE TABLE wiki_mush (
    name varchar(200) PRIMARY KEY,
    hymeniumType varchar(100),
    capShape varchar(100),
    whichGills varchar(100),
    stipeCharacter varchar(100),
    sporePrintColor varchar(100),
    ecologicalType varchar(100),
    howEdible varchar(100)
    );
"""

In [5]:
query_helper.create_table(mushroom_table)

Creating a new table
already exists.


## Go through list of links, get data and commit to db

In [6]:
incomplete_data_list =[]


In [7]:
# list_of_links = ["https://en.wikipedia.org/wiki/Agaricus_abruptibulbus"]
# incomplete_data_list =[]

# for link in list_of_links:
#     raw = webscrape_wiki_table.get_mush_table(list_of_links)
#     #if data not complete save and go to next link
#     if type(raw[0])!= dict:
#         incomplete_data_list.append(raw)
#         continue
#     query_helper.mush_to_tup(raw)


### In future will have some sort of iteration for cellabove

In [8]:
# while web_page_iterator <64:
#     #scrape web infor for 1 page
#     list_of_dict = payscale_uni_webscrape.get_pages(web_page_iterator)

#     #put infor into tuples and commit to db
#     query_helper.tuple_payscale_uni(list_of_dict)
    
#     #iterate and loop
#     web_page_iterator +=1

### Get links from Template:Mycomorphbox wiki page

In [9]:
links = ["https://en.wikipedia.org/w/index.php?title=Special:WhatLinksHere/Template:Mycomorphbox&namespace=0&limit=500",
         "https://en.wikipedia.org/w/index.php?title=Special:WhatLinksHere/Template:Mycomorphbox&namespace=0&limit=500&from=23843036&back=0",
         "https://en.wikipedia.org/w/index.php?title=Special:WhatLinksHere/Template:Mycomorphbox&namespace=0&limit=500&from=48473688&back=23843036"]


In [10]:
def scrape_names(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    info = soup.find(id="mw-whatlinkshere-list")
    links = info.find_all('a')
    
    #get names to add to each mushroom page link
    name = []
    for i in range(0, len(links),3):
        name.append(links[i].get_text())
    #append names to link
    list_of_links =[]
    for i in name:
        form_name = i.replace(" ","_")
        list_of_links.append('https://en.wikipedia.org/wiki/{}'.format(form_name))
    
    #scrape each individual mushroom page for deets
    for link in list_of_links:
        raw = webscrape_wiki_table.get_mush_table(link)
        #if data not complete save and go to next link
        if type(raw[0])!= dict:
            incomplete_data_list.append(raw)
            continue
        #else data is clean so commit to db
        query_helper.mush_to_tup(raw)
        
    
    

In [11]:
# We will pass all links to the scrape_names function
Timer.start()
for link in links:
    scrape_names(link)
Timer.end()

590.219 seconds elapsed


## Get elements with incomplete data and store to analyze later

In [21]:
incomplete_data_list

[['Marasmius',
  'gills on hymenium',
  'cap is convex',
  'hymenium is adnate',
  'stipe is bare',
  'spore print is white',
  'ecology is saprotrophic'],
 ['Phallus indusiatus',
  'glebal hymenium',
  'cap is conical',
  'spore print is olive',
  'ecology is saprotrophic',
  'edibility: choice'],
 ['Pleurotus', 'edibility: choice'],
 ['Hericium erinaceus',
  'teeth on hymenium',
  'no distinct cap',
  'lacks a stipe',
  'spore print is white',
  'ecology is parasitic',
  'edibility: choice'],
 ['Chalciporus piperatus',
  'pores on hymenium',
  'cap is convex',
  'hymenium is adnate',
  'stipe is bare',
  'spore print is brown',
  'edibility: edible, but unpalatable'],
 ['Mycena',
  'gills on hymenium',
  'cap is conical',
  'hymenium is adnate',
  'stipe is bare',
  'spore print is white',
  'ecology is saprotrophic'],
 ['Suillus grevillei',
  'pores on hymenium',
  'cap is convex',
  'stipe has a ring',
  'spore print is ochre',
  'ecology is mycorrhizal',
  'edibility: edible'],
 [

In [ ]:
#important must keep
#set type for sorting below
bs4_type = incomplete_data_list[0][1]
bs4_type

In [ ]:
data_without_hyper = []
names_data_miss = []
flag = False

In [ ]:
for i in incomplete_data_list:
    for j in i:
        if type(j)== type(bs4_type):
            flag =True
            break
    if flag:        
        names_data_miss.append(i[0])
        flag = False
    else:
        data_without_hyper.append(i)

In [ ]:
len(data_without_hyper)

In [ ]:
len(names_data_miss)

In [ ]:
#write all incomplete data to json file
with open('data_missing.json', 'w', encoding='utf-8') as f:
    json.dump(data_without_hyper, f, ensure_ascii=False, indent=4)

In [ ]:
#write all incomplete data to json file
with open('names_with_data_corrupt.json', 'w', encoding='utf-8') as f:
    json.dump(names_data_miss, f, ensure_ascii=False, indent=4)

In [ ]:
#testing cell
web_address = 'https://en.wikipedia.org/wiki/{}'.format(names_data_miss[0])
page = requests.get(web_address)
soup = BeautifulSoup(page.content, 'html.parser')

### IF IMPORT IS MESSED UP LOOK HERE. ASSUMES mush Box is always second
info = soup.find_all('table',{"class":"infobox"})

#get mushroom name 
name = [i.string for i in info[1].findAll('i')]
print(name)
#get mushroom info
divs = info[1].findAll("td")
stuff= divs[1:]
text = []
for i in stuff:
    text.append(i.get_text())

print(text)
name_and_info = name+text

print(name_and_info)

#catch if only name is missing
if (len(text)==7):
    name = [web_address.split('/')[-1].replace('_', ' ')]
#     print(name)
    name_and_info = name+text
#     print(name_and_info)

#catch incomplete values and return them seperate and add them to a list
if(len(name_and_info)!=8):
    name = [web_address.split('/')[-1].replace('_', ' ')]
    print(name)
    name_and_info =name+text
    print(name_and_info)

name_and_info

In [12]:
hymeniumType =  ['gills'   
                ,'pores'
                ,'smooth'
                ,'ridges'
                ,'teeth'
                ,'gleba']

In [13]:
capShape =  ['campanulate'
            ,'conical'
            ,'convex'
            ,'depressed'
            ,'flat'
            ,'infundibuliform'
            ,'offset'
            ,'ovate'
            ,'umbilicate'
            ,'umbonate'
            ,'no'
            ,'NA']

In [14]:
whichGills = ['adnate'
            ,'adnexed'
            ,'decurrent'
            ,'emarginate'
            ,'free'
            ,'seceding'
            ,'sinuate'
            ,'subdecurrent'
            ,'no'
            ,'NA']

In [15]:
stipeCharacter = ['bare'
                ,'ring'
                ,'volva'
                ,'ring_and_volva'
                ,'cortina'
                ,'NA']

In [16]:
sporePrintColor = ['black'
                    ,'blackish-brown'
                    ,'brown'
                    ,'buff'
                    ,'cream'
                    ,'green'
                    ,'ochre'
                    ,'olive'
                    ,'olive-brown'
                    ,'pink'
                    ,'pinkish-brown'
                    ,'purple'
                    ,'purple-black'
                    ,'purple-brown'
                    ,'reddish-brown'
                    ,'salmon'
                    ,'tan'
                    ,'white'
                    ,'yellow'
                    ,'yellow-orange'
                    ,'yellow-brown']

In [19]:
ecologicalType =['mycorrhizal'
                ,'parasitic'
                ,'saprotrophic']

In [18]:
howEdible = ['choice'
            ,'edible'
            ,'inedible'
            ,'unpalatable'
            ,'caution'
            ,'psychoactive'
            ,'poisonous'
            ,'allergenic'
            ,'deadly'
            ,'unknown']

In [ ]:
page = requests.get("https://en.wikipedia.org/w/index.php?title=Special:WhatLinksHere/Template:Mycomorphbox&namespace=0&limit=500")
soup = BeautifulSoup(page.content, 'html.parser')
                       

In [ ]:
info = soup.find(id="mw-whatlinkshere-list")
info

In [ ]:
x =[2,3]
type(x)

In [ ]:
links = info.find_all('a')
links

In [ ]:
name = []
length = len(links)
for i in range(0, len(links),3):
    name.append(links[i].get_text())

In [ ]:
name


In [ ]:

#create a list of links
# name[0].replace(' ', '_')
list_of_links =[]
for i in name:
    form_name = i.replace(" ","_")
    list_of_links.append('https://en.wikipedia.org/wiki/{}'.format(form_name))
    
    
list_of_links

In [ ]:
isinstance(x, list)

In [ ]:
for link in list_of_links:
    print(link)
    raw = webscrape_wiki_table.get_mush_table(link)
    print('success')
    #if data not complete save and go to next link
    if type(raw[0])!= dict:
        incomplete_data_list.append(raw)
        continue
    query_helper.mush_to_tup(raw)

In [ ]:
type(data_without_hyper[14][1])== type(bs4_type)

In [ ]:
raw = webscrape_wiki_table.get_mush_table(list_of_links[0])
raw

In [ ]:
isinstance(type(data_without_hyper[14][1]), bs4_type)

In [ ]:
len(incomplete_data_list)

In [ ]:
total =0
for i in incomplete_data_list:
    total += len(i)
total/len(incomplete_data_list)

In [ ]:
import json
with open('incomplete_data_1209.json', 'w', encoding='utf-8') as f:
    json.dump(incomplete_data_list, f, ensure_ascii=False, indent=4)

In [ ]:
page = requests.get("https://en.wikipedia.org/w/index.php?title=Special:WhatLinksHere/Template:Mycomorphbox&namespace=0&limit=500")
soup = BeautifulSoup(page.content, 'html.parser')
                       

## Wiki Scraping

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/Agaricus_abruptibulbus")
soup = BeautifulSoup(page.content, 'html.parser')
# bodycontent = soup.find(id='bodyContent')
# print(bodycontent)

### IF IMPORT IS MESSED UP LOOK HERE. ASSUMES mush Box is always second
info = soup.find_all('table',{"class":"infobox"})
print(info[1])

# tab = soup.find("table",{"class":"wikitable sortable"})

#### Get species and genus

In [ ]:
name = [i.string for i in info[1].findAll('i')]
name

## Get remaining mushroom info

In [ ]:
divs = info[1].findAll("td")

#remove first line since we dont cate about it then get info
stuff= divs[1:]
text = []
for i in stuff:
    text.append(i.get_text())
#     print(text,'\n')
text

In [ ]:
name_and_info = name+text
print(name_and_info)

In [ ]:
#turn into webscrape function 
#def get_pages(web_page_iterator):
mushroom_dict ={
    'name'          :name_and_info[0],
    'hymeniumType'  :name_and_info[1],
    'capShape'      :name_and_info[2],
    'whichGills'    :name_and_info[3],
    'stipeCharacter':name_and_info[4],
    'sporePrintColor':name_and_info[5],
    'ecologicalType':name_and_info[6],
    'howEdible'      :name_and_info[7]
}


In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/Agaricus_abruptibulbus")
soup = BeautifulSoup(page.content, 'html.parser')
# bodycontent = soup.find(id='bodyContent')
# print(bodycontent)

### IF IMPORT IS MESSED UP LOOK HERE. ASSUMES mush Box is always second
info = soup.find_all('table',{"class":"infobox"})

#get name 
name = [i.string for i in info[1].findAll('i')]

#get mushroom info
divs = info[1].findAll("td")

#remove first line since we dont cate about it then get info
stuff= divs[1:]
text = []
for i in stuff:
    text.append(i.get_text())

name_and_info = name+text

#store into dict and return
mushroom_dict ={
    'name'          :name_and_info[0],
    'hymeniumType'  :name_and_info[1],
    'capShape'      :name_and_info[2],
    'whichGills'    :name_and_info[3],
    'stipeCharacter':name_and_info[4],
    'sporePrintColor':name_and_info[5],
    'ecologicalType':name_and_info[6],
    'howEdible'      :name_and_info[7]
}

In [ ]:
mushroom_dict


In [ ]:
while web_page_iterator <64:
    #scrape web infor for 1 page
    list_of_dict = payscale_uni_webscrape.get_pages(web_page_iterator)

    #put infor into tuples and commit to db
    query_helper.tuple_payscale_uni(list_of_dict)
    
    #iterate and loop
    web_page_iterator +=1
    


In [ ]:
#column names for data file
colnames = ["class",
            "cap_shape",
            "cap_surface",
            "cap_color",
            "bruises",
            "odor",
            "gill_attachment",
            "gill_spacing",
            "gill_size",
            "gill_color",
            "stalk_shape",
            "stalk_root",
            "stalk_surface_above_ring",
            "stalk_surface_below_ring",
            "stalk_color_above_ring",
            "stalk_color_below_ring",
            "veil_type",
            "veil_color",
            "ring_number",
            "ring_type",
            "spore_print_color",
            "population",
            "habitat"]

In [ ]:
df = pd.read_csv('expanded', names=colnames, header=None)#,sep='\t')
df.drop(df.index[-1], inplace=True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df['class'].value_counts()

In [ ]:
df.isna().sum()

In [ ]:
#look at unique items in each count to get a better understanding of data and check for errors
unique_vals =0
for col in colnames:
    unique_vals += len(df[col].value_counts())
    print(df[col].value_counts())
print(unique_vals)

#### First we need to make dummy variables for our catagorical data

In [ ]:
xdf = df.drop(columns=['class', 'odor'])
xdf.head()

In [ ]:
xdf.columns

In [ ]:
# Create dummy variables
data = pd.get_dummies(df.drop(columns=['class', 'odor']))
y = df['class']
data.head()

In [ ]:
#break up data for testing and training
data_train, data_test, y_train, y_test = train_test_split(data, y,test_size=.2)

In [ ]:
#function used to plot feature importance
def plot_feature_importances(model):
    n_features = data_train.shape[1]
    plt.figure(figsize=(20,20))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), data_train.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

## Regular Tree

In [ ]:
# Instantiate and fit a DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(criterion='gini', max_depth=2) 
tree_clf.fit(data_train, y_train)

In [ ]:
# visualizing the regression tree
dot_data = StringIO()
export_graphviz(tree_clf, out_file=dot_data,  
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
plot_feature_importances(tree_clf)

In [ ]:
# Test set predictions
pred = tree_clf.predict(data_test)

# Confusion matrix and classification report
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

## Random Forest

In [ ]:
# Instantiate a BaggingClassifier
# bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=5), 
#                                  n_estimators=20)

In [ ]:
# Instantiate and fit a RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, max_depth= 5)
forest.fit(data_train, y_train)

In [ ]:
# Training accuracy score
forest.score(data_train, y_train)

In [ ]:
# Test accuracy score
forest.score(data_test, y_test)

In [ ]:
plot_feature_importances(forest)

### Load in X and y variables

## Mushroom Observer Database

In [ ]:
dfm = pd.read_csv('observations.csv')
dfm.head()